# Build the environment before running the code

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [2]:
!pip install pytorch-pretrained-bert

    100% |████████████████████████████████| 51kB 3.4MB/s 


In [4]:
from google.colab import files
uploaded = files.upload()

Saving test1.txt to test1.txt
Saving test2.txt to test2.txt
Saving test3.txt to test3.txt
Saving test4.txt to test4.txt
Saving test5.txt to test5.txt
Saving test6.txt to test6.txt
Saving test7.txt to test7.txt
Saving test8.txt to test8.txt
Saving test9.txt to test9.txt
Saving test10.txt to test10.txt


In [5]:
!ls
!pwd

sample_data  test1.txt	test3.txt  test5.txt  test7.txt  test9.txt
test10.txt   test2.txt	test4.txt  test6.txt  test8.txt
/content


# Bert Pytorch- Question and Answer (Edited)

# WORD2VEC gensim

In [6]:
!pip3 install --upgrade gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.6.0)


In [0]:
import gensim.downloader as api
info = api.info()
model = api.load("word2vec-google-news-300")

similarity = model.similarity('woman','fingers')
print(similarity)
print(type(similarity))
print(similarity.item())
print(type(similarity.item()))

[=================================================-] 99.6% 1656.2/1662.8MB downloaded


In [7]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertForMaskedLM
import gensim.downloader as api
import time
import sys

#load data
desk = input("Please input the location of the data file: "'\n')
filename = input("Please input the name of the data file: "'\n')
f = open(desk + filename, "r",encoding='UTF-8')
list = f.readlines()[0:21]

start_time = time.time()

for i in range(len(list)):
    list[i] = list[i].strip()
list[20] = list[20].split('\t',3)
list[20][0] = list[20][0].replace("XXXXX","_")
pre_text = ''
for i in range(20):
    pre_text+= list[i]+' '
target_text = list[20][0]
choices = list[20][3].split("|")

print(pre_text,'\n',target_text,'\n',choices,'\n')

# Load pre-trained model with masked language model head
bert_version = 'bert-large-uncased'
model = BertForMaskedLM.from_pretrained(bert_version)

# Preprocess text
text = pre_text + target_text
# Prevent RuntimeError
if len(text)>2000:
  pre_text = ''
  for i in range(10,20):
    pre_text += list[i]+' '
  text = pre_text + target_text
  print('After decreasing the sentences... ''\n')

tokenizer = BertTokenizer.from_pretrained(bert_version)
tokenized_text = tokenizer.tokenize(text)
mask_positions = []
for i in range(len(tokenized_text)):
    if tokenized_text[i] == '_':
        tokenized_text[i] = '[MASK]'
        mask_positions.append(i)

# Predict missing words from left to right
model.eval()
predicted_token = ''
for mask_pos in mask_positions:
    # Convert tokens to vocab indices
    token_ids = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([token_ids])
    # print('tokens_tensor: ''\n',tokens_tensor)
    # Call BERT to predict token at this position
    try:
      predictions = model(tokens_tensor)[0, mask_pos]
    except RuntimeError:
      sys.exit('Oops! Sorry for your input 1-20 articles are too long. Try to decrease your sentences.')
      # print('Oops! Sorry for your input 1-20 articles are too long. Try to decrease your sentences.')
      # break
    else:
      predictions = model(tokens_tensor)[0, mask_pos]
    # print("type.predictions:",type(predictions))
    # print("predictions:"'\n',predictions)
    predicted_index = torch.argmax(predictions).item()
    # print('predicted_index''\n',predicted_index)
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
    print('predicted_token:''\n',predicted_token)
    
    # for i in range(10):
    #   predicted_token1 = tokenizer.convert_ids_to_tokens([predicted_index-i])[0]
    #   print('predicted_token(+1):',predicted_token1)
    
    # Update text
    tokenized_text[mask_pos] = predicted_token

for mask_pos in mask_positions:
    tokenized_text[mask_pos] = "_" + tokenized_text[mask_pos] + "_"
result = ' '.join(tokenized_text).replace(' ##', '').replace(pre_text,'')

result21 = target_text.replace('_',tokenized_text[mask_pos])
print('After predicting: ''\n', result21)

answer_number = -1
for answer_index in range(10):
  # use word2vec to find the most similar answer if the prediction is not in the selection
  if  predicted_token in choices:
    if predicted_token == choices[answer_index]:
      answer_number = answer_index
      break
  else:
    info = api.info()
    model = api.load("word2vec-google-news-300")
    similarity_list = []
    for similarity_index in range(10):
      try:
        similarity = model.similarity(predicted_token,choices[similarity_index])
      except KeyError:
        similarity = 0.0000000001
      else:
        similarity = model.similarity(predicted_token,choices[similarity_index])
        similarity = similarity.item() #.item(): convert np.float to float type
      similarity_list.append(similarity)
    print('similarity_list: ''\n',similarity_list)
    most_similar = max(similarity_list)
    most_similar_index = similarity_list.index(max(similarity_list))
    print('Most similar answer is: ''\n',most_similar_index+1,' ',choices[most_similar_index],' ',most_similar)
    
    answer_number = most_similar_index
    predicted_token = choices[most_similar_index]
    break
    
print('The Answer is: ''\n', answer_number+1,' ', predicted_token)

print("--- %s seconds ---" % (time.time() - start_time))

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
1 Once upon a time, there lived a farmer in a village, beside a forest. 2 He had a big garden that had an old apple tree and other plants, trees and beautiful flowers. 3 When the farmer was a little boy, he spent much of his time playing with the apple tree. Those days, the apple tree had given the choicest of apples to him. 4 However, as time passed, the apple tree became old and stopped bearing fruits. 5 Now that the farmer was not getting any apples from the tree, he decided that the tree was useless. 6 Therefore, he decided to cut the tree and use its wood to make some new furniture. 7 He felt that since the tree was old and huge, he did not have to cure it, and it would make great furniture. 8 He forgot that as a boy, he had spent his entire childhood climbing the tree and eating its apples. 9 Now the apple tree was home to several little animals in the neighborhood. 10 This included squirrel

100%|██████████| 231508/231508 [00:00<00:00, 444466.74B/s]


predicted_token:
 get
After predicting: 
 21 All the little animals became desperate, and wanted to _get_ the apple tree at any cost.
[================================------------------] 64.8% 1077.4/1662.8MB downloadedBuffered data was truncated after reaching the output size limit.